In [18]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Loading results dataframes for use in linear regression modeling
run_data = pd.read_csv("data/stream-data/raw-stream-data/russian.csv")
days_below_32 = pd.read_csv("data/weather-data/KenaiAirport_Days-Below-32F.csv")
summer_mean_temps = pd.read_csv("data/weather-data/KenaiAirport_Summer-Mean-Temps-Yearly.csv")
winter_mean_temps = pd.read_csv("data/weather-data/KenaiAirport_Winter-Mean-Temps-Yearly.csv")

Now that we have our data loaded and the proper modules imported, let's process the Russian
River run datafile to parse out our target output variables (run counts) as well as gather a 
few more potential input variables (effort counts or "angler days").

In [19]:
drop_columns = ['ER  Escapement', 'LR Escapement', 'ER', 'LR']
run_data.drop(columns=drop_columns, inplace=True)
run_data

,Year,Effort,ER Sport,LR Sport,ER Local,LR Local,Total Run
0,1962,ND,"3,408","1,292","32,388","20,526","52,914"
1,1963,"7,880","3,670","1,390","18,050","52,510","70,560"
2,1964,"5,330","3,550","2,450","16,250","49,380","65,630"
3,1965,"9,720","10,030","2,160","31,544","23,980","55,524"
4,1966,"18,280","14,950","7,290","31,608","41,720","73,328"
5,1967,"16,960","7,240","5,720","20,950","55,200","76,150"
6,1968,"17,280","6,920","5,820","16,112","54,700","70,812"
7,1969,"14,930","5,870","1,150","10,870","30,022","40,892"
8,1970,"10,700","5,750",600,"11,201","26,800","38,001"
9,1971,"15,120","2,810","10,730","5,464","65,151","70,615"
